# Data preprocessing

## Importing required libraries and reading the input json files:

In [1]:
import os, datetime, json
import pandas as pd
import numpy as np

In [2]:
transcript = pd.read_json('../data/transcript.json', orient='records', lines=True)
portfolio = pd.read_json('../data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('../data/profile.json', orient='records', lines=True)

## Data cleansing operations

### Cleaning profile.json

In [3]:
profile.dropna(axis=0,inplace=True)
profile.rename(columns={'id': 'cust_id'}, inplace=True)
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'].astype(str), format='%Y%m%d')
profile['days_as_member'] = (datetime.datetime.today() - profile['became_member_on']).dt.days
profile.sample(20)

,gender,age,cust_id,became_member_on,income,days_as_member
1303,F,62,690ce015f65549f693d69593f3bd4663,2016-06-13,107000.0,1624
6743,M,18,2f31ca6bc8f741f2b1a809a38971d6bb,2018-07-25,40000.0,852
12111,F,58,490ca12f690d413aaf75d08c56067ea4,2017-04-23,111000.0,1310
6542,M,75,f496c159664c42dca6fff0c428ef5973,2016-06-28,85000.0,1609
7142,F,88,8142c6f808944e7a87a18950c71b18ae,2017-06-07,54000.0,1265
3797,F,56,7b32d67348b6462fad4f0f59ee3bd8af,2017-06-06,118000.0,1266
16407,M,50,87cd8024319544c18c17e3045ddfd83a,2016-11-25,91000.0,1459
13828,M,48,54ce16dc2b7b4fbb8d540a7d1161f92e,2014-06-27,48000.0,2341
9400,M,77,680eaabd027747e88cf77ac89a978778,2017-10-02,75000.0,1148
11919,O,25,d499c6d63b884b83b3c06acf4fe8bd7b,2018-02-01,32000.0,1026


In [4]:
profile.to_csv('../data/cln_profile.csv')

### Cleaning portfolio.json

In [5]:
portfolio.rename(columns={'id': 'offer_id'}, inplace=True)
portfolio.set_index('offer_id', inplace=True)
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, ae264e3637204a6fb9bb56bc8210ddfd to 2906b810c7d4411798c6938adc9daaa5
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
dtypes: int64(3), object(2)
memory usage: 480.0+ bytes


#### Unpacking channels

In [6]:
medium = portfolio['channels'].str.join(',').str.get_dummies(',')
portfolio = pd.concat([portfolio, medium], axis=1) # concat medium columns
portfolio = portfolio.drop(['channels'], axis=1)
portfolio['duration_in_hours'] = portfolio['duration']*24
portfolio

,reward,difficulty,duration,offer_type,email,mobile,social,web,duration_in_hours
offer_id,,,,,,,,,
ae264e3637204a6fb9bb56bc8210ddfd,10,10,7,bogo,1,1,1,0,168
4d5c57ea9a6940dd891ad53e9dbe8da0,10,10,5,bogo,1,1,1,1,120
3f207df678b143eea3cee63160fa8bed,0,0,4,informational,1,1,0,1,96
9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1,1,0,1,168
0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1,0,0,1,240
2298d6c36e964ae4a3e7e9706d1fb8c2,3,7,7,discount,1,1,1,1,168
fafdcd668e3743c1bb461111dcafc2a4,2,10,10,discount,1,1,1,1,240
5a8bc65990b245e5a138643cd4eb9837,0,0,3,informational,1,1,1,0,72
f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,bogo,1,1,1,1,120


In [7]:
portfolio.to_csv('../data/cln_portfolio.csv')

### Cleaning and grouping transcript.json based on cust_id, event

In [8]:
transcript.rename(columns={'person': 'cust_id'}, inplace=True)
transcript.rename(columns={'time':'hours_till_action'},inplace=True)
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   cust_id            306534 non-null  object
 1   event              306534 non-null  object
 2   value              306534 non-null  object
 3   hours_till_action  306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


#### Unpacking value 

In [9]:
# parse the json format of value column to individual columns
transcript = pd.concat([transcript, transcript['value'].apply(pd.Series)], axis=1)

# offer_id shows in two separate columns due to the two different key types. 
# create new column to combine the two offer_id columns
transcript['offer_id_new'] = np.where(transcript['offer id'].isnull() & transcript['offer_id'].notnull(),transcript['offer_id'],transcript['offer id'])

#drop unnecessary offer_id columns
transcript.drop(['offer id','offer_id'],axis=1,inplace=True)
transcript.drop(['value'],axis=1,inplace=True)

#rename offer_id column
transcript.rename(columns={'offer_id_new':'offer_id'},inplace=True)
transcript.sample(10)

,cust_id,event,hours_till_action,amount,reward,offer_id
26475,18f6d43ce45c41b398606753c3412f60,offer viewed,36,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d
20998,b50987e063ec45fca2b6343837d6a139,transaction,18,14.97,NaN,NaN
116166,933bab17f8d24bbca0971d1c1877ed33,offer received,336,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837
229118,187bc1f183174e6799994488d7b2bf2b,offer completed,528,NaN,10.0,4d5c57ea9a6940dd891ad53e9dbe8da0
61885,2e4954fe70524b6dbaa24816e040ab42,offer received,168,NaN,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2
87364,b0c6b9ecae7f47dc93a44edef68ab1a8,transaction,222,11.48,NaN,NaN
78855,89d8a13cb46b40c59407f5c02b1a8696,offer viewed,192,NaN,NaN,3f207df678b143eea3cee63160fa8bed
165821,e4a75f2ce99b45e78d8b42bfb31b2ef4,offer completed,408,NaN,5.0,9b98b8c7a33c4b65b9aebfe6a799e6d9
58150,84ea71fb031940868ed5fbe70fc67edd,offer received,168,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0
256502,291b4bd6461c42ff8cac640f351a2c70,offer received,576,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0


In [10]:
transcript.to_csv('../data/cln_transcript.csv')

### Creating total df

In [11]:
total = transcript.merge(portfolio,how='left',on='offer_id')
total = total.merge(profile,how='left',on='cust_id')
total.rename(columns={'reward_x':'reward_received', 'reward_y':'reward_defined'},inplace=True)
total.sample(10)

,cust_id,event,hours_till_action,amount,reward_received,offer_id,reward_defined,difficulty,duration,offer_type,email,mobile,social,web,duration_in_hours,gender,age,became_member_on,income,days_as_member
116906,23ce738b421d4731a8f8c09dea537aea,offer received,336,NaN,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,7.0,7.0,discount,1.0,1.0,1.0,1.0,168.0,F,32.0,2018-04-04,54000.0,964.0
223773,81f18e0b118342069adfe3c16be8dc2e,offer completed,516,NaN,3.0,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,7.0,7.0,discount,1.0,1.0,1.0,1.0,168.0,M,63.0,2017-12-31,50000.0,1058.0
261791,2d54fa98743946f286e28acfd24cd4cd,offer viewed,576,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0,10.0,7.0,discount,1.0,1.0,0.0,1.0,168.0,NaN,NaN,NaT,NaN,NaN
110029,77b27d5f09db42cf9c844ab58d132e85,transaction,330,14.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,81.0,2018-03-15,89000.0,984.0
86733,5aa466a27dfd48e78c95f6ba3ce5fd8e,transaction,216,4.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,27.0,2015-12-12,58000.0,1808.0
169992,c2a5c2407540466b9b0f7e8e50715a73,transaction,414,31.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,49.0,2016-11-17,98000.0,1467.0
81037,3fab840fa8cd489faf5b86ae7e49b340,offer viewed,198,NaN,NaN,ae264e3637204a6fb9bb56bc8210ddfd,10.0,10.0,7.0,bogo,1.0,1.0,1.0,0.0,168.0,M,36.0,2018-03-09,33000.0,990.0
160369,3b346dd87f574892b5080d676b4e69ff,offer received,408,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837,0.0,0.0,3.0,informational,1.0,1.0,1.0,0.0,72.0,M,38.0,2017-07-08,56000.0,1234.0
105584,7028c194e0f443829e3049a584abd56a,transaction,300,3.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
130270,4ff9be4a461b4130ae28887591c15915,offer viewed,348,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4,2.0,10.0,10.0,discount,1.0,1.0,1.0,1.0,240.0,F,50.0,2018-03-25,110000.0,974.0


#### Creating offer expired label

In [12]:
total[total['hours_till_action'] > total['duration_in_hours']]

,cust_id,event,hours_till_action,amount,reward_received,offer_id,reward_defined,difficulty,duration,offer_type,email,mobile,social,web,duration_in_hours,gender,age,became_member_on,income,days_as_member
36952,1ccdd48ea41247248269cb0c5805ca12,offer viewed,78,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837,0.0,0.0,3.0,informational,1.0,1.0,1.0,0.0,72.0,F,83.0,2017-12-23,99000.0,1066.0
36981,b5c5091888604fefb1219e5fa0aece97,offer viewed,78,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837,0.0,0.0,3.0,informational,1.0,1.0,1.0,0.0,72.0,M,71.0,2017-05-02,87000.0,1301.0
36987,5137dcb4eff644888b63af6dcaf8b560,offer viewed,78,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837,0.0,0.0,3.0,informational,1.0,1.0,1.0,0.0,72.0,M,59.0,2016-04-08,64000.0,1690.0
37004,422a5be8f91e4a65854b715254db72e4,offer viewed,78,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837,0.0,0.0,3.0,informational,1.0,1.0,1.0,0.0,72.0,M,48.0,2016-02-06,52000.0,1752.0
37068,2b6d8eb54c964e73aa022301f51a621a,offer viewed,78,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837,0.0,0.0,3.0,informational,1.0,1.0,1.0,0.0,72.0,F,52.0,2017-03-16,42000.0,1348.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,714,NaN,3.0,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,7.0,7.0,discount,1.0,1.0,1.0,1.0,168.0,NaN,NaN,NaT,NaN,NaN
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,714,NaN,2.0,fafdcd668e3743c1bb461111dcafc2a4,2.0,10.0,10.0,discount,1.0,1.0,1.0,1.0,240.0,NaN,NaN,NaT,NaN,NaN
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer viewed,714,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,5.0,20.0,10.0,discount,1.0,0.0,0.0,1.0,240.0,F,60.0,2017-09-08,64000.0,1172.0
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,714,NaN,2.0,fafdcd668e3743c1bb461111dcafc2a4,2.0,10.0,10.0,discount,1.0,1.0,1.0,1.0,240.0,M,39.0,2018-06-27,39000.0,880.0


In [7]:
completes = transcript.loc[(transcript['event'] == "offer completed")]

completes['offer_completed_id'] = completes['value'].apply(lambda x: list(x.values())[0])
completes['offer_completed_reward'] = completes['value'].apply(lambda x: list(x.values())[1])
completes['offer_duration'] = completes['offer_completed_id'].apply(lambda x: offer_dict[x]['duration'])


indexes = completes[completes['days'] > completes['offer_duration']].index
transcript['event'][indexes] = "offer expired"


<ipython-input-7-0d0eb6862d44>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completes['offer_completed_id'] = completes['value'].apply(lambda x: list(x.values())[0])
<ipython-input-7-0d0eb6862d44>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completes['offer_completed_reward'] = completes['value'].apply(lambda x: list(x.values())[1])
<ipython-input-7-0d0eb6862d44>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [11]:
transcript['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [12]:
transcript.to_csv('../data/cln_transcript.csv')

In [10]:
for label, group in transcript.groupby(['cust_id', 'event']):
    custid = label[0]
    event_name = label[1]
    if (event_name == "offer completed"):
        profile.loc[profile['cust_id'] == custid, 'offers_completed'] = len(group)
    elif (event_name == "offer received"):
        profile.loc[profile['cust_id'] == custid, 'offers_received'] = len(group)
    elif (event_name == "offer viewed"):
        profile.loc[profile['cust_id'] == custid, 'offers_viewed'] = len(group)

### Adding features to profile

#### Total of all events

In [13]:
total['count_of_events'] = 1
count_of_events = pd.pivot_table(total, values='count_of_events', index = 'cust_id', columns='event', aggfunc=np.sum)
count_of_events.rename(columns=lambda x: "Total "+x, inplace=True)

profile_features = profile
profile_features = profile_features.merge(count_of_events,how='left',on='cust_id')

#### Individual event counts based on offer type

In [14]:
event_group = total.groupby('event')
offer_counts = dict()

for event, group in event_group:
    if event != 'transaction':        
        group['offer_counts'] = 1
        offer_counts[event] = pd.pivot_table(group, values='offer_counts', index='cust_id', columns='offer_type', aggfunc=np.sum)
        offer_counts[event].rename(columns=lambda x: event+" "+x+" count", inplace=True)
        
    else:
        offer_counts[event] = group.groupby(['cust_id'])['amount'].agg('sum').to_frame()
        offer_counts[event].rename(columns=lambda x: "Total transaction "+x, inplace=True)
    profile_features = profile_features.merge(offer_counts[event],how='left',on='cust_id')

profile_features.fillna(0, inplace=True)    
profile_features

<ipython-input-14-e5bda018f29c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['offer_counts'] = 1


,gender,age,cust_id,became_member_on,income,days_as_member,Total offer completed,Total offer received,Total offer viewed,Total transaction,offer completed bogo count,offer completed discount count,offer received bogo count,offer received discount count,offer received informational count,offer viewed bogo count,offer viewed discount count,offer viewed informational count,Total transaction amount
0,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,1227,1.0,2.0,0.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,77.01
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,1294,3.0,4.0,4.0,7.0,3.0,0.0,3.0,0.0,1.0,3.0,0.0,1.0,159.27
2,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,942,2.0,4.0,3.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,57.73
3,M,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,1018,5.0,6.0,6.0,3.0,3.0,2.0,4.0,2.0,0.0,4.0,2.0,0.0,36.43
4,M,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,1108,1.0,3.0,2.0,4.0,0.0,1.0,0.0,2.0,1.0,0.0,2.0,0.0,15.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0,903,0.0,3.0,3.0,7.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,20.03
14821,M,61,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0,864,1.0,3.0,1.0,7.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,25.97
14822,M,49,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0,1397,0.0,3.0,1.0,8.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,39.74
14823,F,83,9dc1421481194dcd9400aec7c9ae6366,2016-03-07,50000.0,1722,3.0,3.0,3.0,14.0,3.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,189.67


In [15]:
profile_features['total_view_ratio'] = round(profile_features['Total offer viewed']/profile_features['Total offer received'], 2)
profile_features['total_completion_ratio'] = round(profile_features['Total offer completed']/profile_features['Total offer received'], 3)
profile_features['avg_transaction_amt'] = round(profile_features['Total transaction amount']/profile_features['Total transaction'], 3)
profile_features.sample(10)

,gender,age,cust_id,became_member_on,income,days_as_member,Total offer completed,Total offer received,Total offer viewed,Total transaction,...,offer received bogo count,offer received discount count,offer received informational count,offer viewed bogo count,offer viewed discount count,offer viewed informational count,Total transaction amount,total_view_ratio,total_completion_ratio,avg_transaction_amt
12908,M,44,be7d070f14a74e61a23ba4acbda84754,2018-01-17,36000.0,1041,0.0,2.0,0.0,4.0,...,0.0,1.0,1.0,0.0,0.0,0.0,12.26,0.00,0.000,3.065
4167,M,53,8b7d6291add449f7812b3435e77bc304,2018-05-17,62000.0,921,2.0,4.0,3.0,13.0,...,1.0,1.0,2.0,1.0,1.0,1.0,222.62,0.75,0.500,17.125
9482,M,60,2a6bb506b41540378b1d373985502233,2017-08-22,91000.0,1189,0.0,6.0,6.0,1.0,...,2.0,3.0,1.0,2.0,3.0,1.0,11.28,1.00,0.000,11.280
4991,F,58,c0bbc13872474c63a83e8b503bb88f72,2015-09-06,85000.0,1905,4.0,4.0,4.0,11.0,...,3.0,1.0,0.0,3.0,1.0,0.0,1075.80,1.00,1.000,97.800
11948,F,56,95f37a4a6f8b4bf6b9f91d62db4457dc,2013-08-18,38000.0,2654,2.0,3.0,2.0,19.0,...,0.0,3.0,0.0,0.0,2.0,0.0,47.69,0.67,0.667,2.510
6278,M,69,beb57037456e4445b9498b9e64d3d3b7,2014-04-29,67000.0,2400,0.0,4.0,2.0,9.0,...,1.0,1.0,2.0,1.0,0.0,1.0,36.46,0.50,0.000,4.051
9608,M,62,1bb6878ed4ed4e5e804829102ad73113,2018-07-26,34000.0,851,1.0,3.0,2.0,4.0,...,1.0,1.0,1.0,1.0,0.0,1.0,23.24,0.67,0.333,5.810
8161,F,80,2d59f307fcbf4f4dab5218893231e522,2017-12-12,82000.0,1077,2.0,5.0,5.0,3.0,...,1.0,4.0,0.0,1.0,4.0,0.0,59.94,1.00,0.400,19.980
13893,M,68,3a16f36ede6d41b8b2ff466de14b32d2,2016-05-15,59000.0,1653,3.0,6.0,6.0,5.0,...,1.0,4.0,1.0,1.0,4.0,1.0,77.79,1.00,0.500,15.558
3716,F,58,e88c17e2c42d432cb8fdb808c85920a2,2018-06-25,100000.0,882,1.0,4.0,3.0,1.0,...,1.0,2.0,1.0,0.0,2.0,1.0,26.16,0.75,0.250,26.160


In [16]:
len(profile_features[profile_features['Total offer viewed'] < profile_features['Total offer completed']])

2005

In [17]:
profile_features.to_csv('../data/xtr_profile.csv')